# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [ ]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('/content/sample_data/previsao_de_renda.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [ ]:
# Removendo coluna Unnamed: 0, data_ref e id_cliente
df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'], inplace=True)

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.
    

In [ ]:
# 1) Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis

# Fórmula para prever log(renda)
formula = 'np.log(renda) ~ C(sexo) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + tempo_emprego + qtd_filhos + idade + qt_pessoas_residencia'

# Gera X (covariáveis) e y (variável dependente)
y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

# Ajustar o modelo de regressão
modelo = sm.OLS(y, X).fit()

# Resumo dos parâmetros
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     310.4
Date:                Thu, 27 Mar 2025   Prob (F-statistic):               0.00
Time:                        15:46:11   Log-Likelihood:                -13591.
No. Observations:               12427   AIC:                         2.723e+04
Df Residuals:                   12404   BIC:                         2.740e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [ ]:
# Mantenha sempre a categoria mais frequente como casela de referência
# Encontrando as variáveis mais frequentes de cada categoria:

print(df['educacao'].value_counts())
print('\n')
print(df['tipo_renda'].value_counts())
print('\n')
print(df['estado_civil'].value_counts())
print('\n')
print(df['tipo_residencia'].value_counts())

educacao
Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: count, dtype: int64


tipo_renda
Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: count, dtype: int64


estado_civil
Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: count, dtype: int64


tipo_residencia
Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comunitário         64
Name: count, dtype: int64


In [ ]:
# Formula atualizada com Treatmente para cada categoria
formula_atualizada = 'np.log(renda) ~ C(educacao, Treatment(reference="Secundário")) + C(sexo) + C(tipo_renda, Treatment(reference="Assalariado")) + C(estado_civil, Treatment(reference="Casado")) + C(tipo_residencia, Treatment(reference="Casa")) + tempo_emprego + qtd_filhos + idade + qt_pessoas_residencia'

In [ ]:
# Gera X (covariáveis) e y (variável dependente)
y, X = patsy.dmatrices(formula_atualizada, data=df, return_type='dataframe')

# Ajustar o modelo de regressão
modelo_at0 = sm.OLS(y, X).fit()

# Resumo do modelo atualizado
print(modelo_at0.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     310.4
Date:                Thu, 27 Mar 2025   Prob (F-statistic):               0.00
Time:                        16:18:39   Log-Likelihood:                -13591.
No. Observations:               12427   AIC:                         2.723e+04
Df Residuals:                   12404   BIC:                         2.740e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [ ]:
# 2) Remova a variável menos significante e analise

# Removendo [T.Com os pais]
df_filtrado_v1 = df[df['tipo_residencia'] != 'Com os pais']

# Gera X (covariáveis) e y (variável dependente)
y, X = patsy.dmatrices(formula_atualizada, data=df_filtrado_v1, return_type='dataframe')

# Ajustar o modelo de regressão
modelo_at1 = sm.OLS(y, X).fit()

# Resumo do modelo atualizado
print(modelo_at1.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     309.3
Date:                Thu, 27 Mar 2025   Prob (F-statistic):               0.00
Time:                        15:49:56   Log-Likelihood:                -12887.
No. Observations:               11753   AIC:                         2.582e+04
Df Residuals:                   11731   BIC:                         2.598e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [ ]:
#3) Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%. Compare o modelo final com o inicial.

# Filtrar removendo múltiplas categorias
df_filtrado_v2 = df[~df['tipo_residencia'].isin(['Com os pais', 'Aluguel']) & ~df['educacao'].isin(['Primário', 'Superior incompleto'])]

# Gera X (covariáveis) e y (variável dependente)
y, X = patsy.dmatrices(formula, data=df_filtrado_v2, return_type='dataframe')

# Ajustar o modelo de regressão
modelo_at_2 = sm.OLS(y, X).fit()

# Resumo do modelo atualizado
print(modelo_at_2.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.359
Method:                 Least Squares   F-statistic:                     342.8
Date:                Thu, 27 Mar 2025   Prob (F-statistic):               0.00
Time:                        16:17:43   Log-Likelihood:                -12077.
No. Observations:               11000   AIC:                         2.419e+04
Df Residuals:                   10981   BIC:                         2.433e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [ ]:
# Removendo C(educacao)[T.Superior incompleto] do df
df_filtrado_v3 = df[
    ~df['tipo_residencia'].isin(['Com os pais', 'Aluguel', 'Comunitário', 'Estúdio', 'Governamental']) &
    ~df['educacao'].isin(['Primário', 'Superior incompleto', 'Pós graduação']) &
    ~df['tipo_renda'].isin(['Bolsista', 'Pensionista', 'Servidor público']) &
    ~df['estado_civil'].isin(['União'])
]

# Gera X (covariáveis) e y (variável dependente)
y, X = patsy.dmatrices(formula_atualizada, data=df_filtrado_v3, return_type='dataframe')

# Ajustar o modelo de regressão
modelo_at_3 = sm.OLS(y, X).fit()

# Resumo do modelo atualizado
print(modelo_at_3.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     486.7
Date:                Thu, 27 Mar 2025   Prob (F-statistic):               0.00
Time:                        16:25:10   Log-Likelihood:                -9502.1
No. Observations:                8693   AIC:                         1.903e+04
Df Residuals:                    8682   BIC:                         1.910e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                                                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

#### Relação de R²

Vamos avaliar o desempenho do R² de cada modelo, de acordo com as variáveis excluidas.

✔️ = R² maior do que o modelo base

|Modelo| R² Obtido| Resultado |
|-|-|-|
|Modelo Base             | R² 0.355 | - |
|df_filtrado_v1          | R² 0.356 | ✔️ |
|df_filtrado_v2          | R² 0.360 | ✔️ |
|df_filtrado_v3          | R² 0.359 | ✔️ |